In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 82% 21.0M/25.7M [00:01<00:00, 29.5MB/s]
100% 25.7M/25.7M [00:01<00:00, 22.1MB/s]


In [7]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [8]:
import pandas as pd

In [9]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [10]:
df = temp_df.iloc[:1000]

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
df.shape

(1000, 2)

In [13]:
df.duplicated().sum()

0

In [14]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [15]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-15-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [16]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-16-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [18]:
!pip install nltk

In [21]:
import nltk

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

<ipython-input-23-9f68b5950da1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [24]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [25]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [27]:
import gensim

In [28]:
from gensim.models import Word2Vec,KeyedVectors

In [31]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

In [32]:
X_train = X_train['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))
# Remove stopwords

X_test = X_test['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [33]:
import spacy
import en_core_web_sm
# Load the spacy model. This takes a few seconds.
nlp = en_core_web_sm.load()
# Process a sentence using the model
doc = nlp(X_train.values[0])
print(doc.vector)

[ 0.10028061 -0.2824126   0.01091258 -0.02692064 -0.11199486  0.07451548
  0.38245845  0.29818484  0.40270728 -0.26509836  0.41719398 -0.14491141
 -0.40324908 -0.27088803 -0.2521854   0.2279773  -0.47554725 -0.5674921
 -0.38171986 -0.1223709   0.08560037  0.55040723 -0.16157518  0.02266293
  0.07961473  0.09753469  0.510546    0.44566435  0.25641567  0.34343463
  0.07518934 -0.02362541 -0.19986375 -0.21560466 -0.01439066  0.20884512
  0.02591227 -0.05159931 -0.04821331  0.0035865  -0.4069838   0.28670862
 -0.06950787  0.2823058   0.11032355 -0.10251382 -0.15382905  0.08040971
  0.08704211  0.17740142 -0.06067834  0.04319293  0.00795529 -0.35006955
 -0.3353541   0.04519325  0.26071352 -0.21468212 -0.01722909 -0.07951423
 -0.3353837  -0.17842022 -0.1778624  -0.00317474  0.13813923 -0.23558648
 -0.06406814 -0.08009888  0.10479124 -0.0527338   0.6718636   0.38819966
  0.3070201  -0.27698374  0.2648724  -0.21414553 -0.17748705 -0.53261316
  0.0357673  -0.5369589  -0.16997866 -0.5567461  -0.

In [34]:
input_arr = []
for item in X_train.values:
    doc = nlp(item)
    input_arr.append(doc.vector)

In [36]:
import numpy as np

In [37]:
input_arr = np.array(input_arr)

In [38]:
input_arr.shape

(800, 96)

In [39]:
input_test_arr = []
for item in X_test.values:
    doc = nlp(item)
    input_test_arr.append(doc.vector)

In [40]:
input_test_arr = np.array(input_test_arr)

In [41]:
input_test_arr.shape

(200, 96)

In [42]:
from sklearn.linear_model import LogisticRegression
algo = LogisticRegression()
algo.fit(input_arr,y_train)

LogisticRegression()

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
y_pred = algo.predict(input_test_arr)
accuracy_score(y_test,y_pred)

0.63

In [48]:
y_test[0]

1

In [49]:
y_pred[0]

1